In [1]:
import cnot_network

from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sbs
import numpy as np
sbs.set_context("paper")
sbs.set_theme()
sbs.set_style("dark")

from surface_code_routing.tikz_utils import tikz, tex

sbs.set_style("whitegrid", {'grid.linestyle': '-.'})

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": "Helvetica",
})

def plt_data(condition, x_axis, y_axis, target=None):
    data = filter(lambda x: x[0] == target, zip(condition, x_axis, y_axis))
    x = []
    y = []
    for _, x_val, y_val in data:
        x.append(x_val)
        y.append(y_val)
    return {'x':x, 'y':y}   

In [2]:
import numpy as np
from functools import partial
from surface_code_routing.dag import DAG
from surface_code_routing.symbol import Symbol
from surface_code_routing.instructions import INIT, CNOT, MEAS, X, Hadamard, X_MULTI
from surface_code_routing.lib_instructions import T_Factory, Toffoli, T_gate

from surface_code_routing.compiled_qcb import compile_qcb

low = 2
high = 10
mct = partial(range, low, high + 1)

def multi_toffoli(n_ctrls, height, width, *externs, init=True, toffoli=Toffoli, **kwargs):

    dag = DAG(
        Symbol(
            f'MCToff_{n_ctrls}',
            tuple([f'ctrl_{i}' for i in range(n_ctrls)] + ['targ']),
            tuple([f'ctrl_{i}' for i in range(n_ctrls)] + ['targ'])
        )
    )
    
    if init:
        for i in range(1, n_ctrls - 1):
            dag.add_gate(INIT(f'anc_{i}'), )

    if n_ctrls == 2:
        dag.add_gate(toffoli('ctrl_0', 'ctrl_1', 'targ'))
        return compile_qcb(dag, height, width, *externs, **kwargs) 
    
    # First Toffoli
    dag.add_gate(toffoli(
        *list(map(dag.__getitem__, 
        ('ctrl_0', 'ctrl_1', 'anc_1')
           ))
    ))

    for i in range(2, n_ctrls - 1):
        dag.add_gate(toffoli(f'anc_{i - 1}', f'ctrl_{i}', f'anc_{i}'))
        
        
    dag.add_gate(toffoli(f'anc_{n_ctrls - 2}', f'ctrl_{n_ctrls - 1}', 'targ'))

    for i in range(n_ctrls - 2,  1, -1):
        dag.add_gate(toffoli(f'anc_{i - 1}', f'ctrl_{i}', f'anc_{i}'))

    dag.add_gate(toffoli('ctrl_0', 'ctrl_1', 'anc_1'))

    return compile_qcb(dag, height, width, *externs, **kwargs) 

In [3]:
import cnot_network
from functools import partial
from surface_code_routing.lib_instructions import T_Factory, Toffoli, T_gate

toff_height = 14
toff_width = toff_height

qcb_size = 32

t_factory_l1 = cnot_network.T_Factory()
toffoli_gate = cnot_network.Toffoli

t_factory_l2 = T_Factory(t_factory_l1, height=8, width=10, t_gate=T_gate(t_factory_l1))
t_gate_l2 = T_gate(factory=t_factory_l2)

t_factory_l3 = T_Factory(t_factory_l2, height=11, width=12, t_gate=T_gate(t_factory_l2))
t_gate_l3 = T_gate(factory=t_factory_l3)

extern_toffoli_l1 = cnot_network.toffoli(toff_height, toff_width, t_factory=t_factory_l1)
extern_toffoli_l2 = cnot_network.toffoli(toff_height, toff_width, t_factory=t_factory_l2)
extern_toffoli_l3 = cnot_network.toffoli(toff_height, toff_width, t_factory=t_factory_l3)

# Raw T injection sites
ccz_factory_l1 = cnot_network.CCZ_factory(6, 4, t_factory=None)
ccz_factory_l2 = cnot_network.CCZ_factory(9, 7, t_factory=t_factory_l1)
ccz_factory_l3 = cnot_network.CCZ_factory(11, 11, t_factory=t_factory_l2)

t_factories = (t_factory_l1, t_factory_l2, t_factory_l3)
extern_toffolis = (extern_toffoli_l1, extern_toffoli_l2, extern_toffoli_l3)
ccz_factories = (ccz_factory_l1, ccz_factory_l2, ccz_factory_l3)

In [4]:
def nested_multi_toffoli(n_ctrls, height, width, *externs, toffoli_gate=Toffoli, multi_toffoli=None, **kwargs):

    dag = DAG(
        Symbol(
            f'MCToff_{n_ctrls}',
            tuple([f'ctrl_{i}' for i in range(n_ctrls)] + ['targ']),
            tuple([f'ctrl_{i}' for i in range(n_ctrls)] + ['targ'])
        )
    )

    def multi_toffoli_gate(*args):
        dag = multi_toffoli.instruction(args=args, targs=args)
        return dag

    n_half_ctrls = n_ctrls // 2
    
    dag.add_gate(INIT(f'targ_a'))
    dag.add_gate(INIT(f'targ_b'))

    # Read-in
    dag.add_gate(multi_toffoli_gate(
        *[f'ctrl_{i}' for i in range(n_half_ctrls)],
        'targ_a')
    )

    dag.add_gate(multi_toffoli_gate(
        *[f'ctrl_{i}' for i in range(n_half_ctrls, n_ctrls)],
        'targ_b')
    )

    # Write to targ
    dag.add_gate(toffoli_gate('targ_a', 'targ_b', 'targ'))

    # Uncompute
    dag.add_gate(multi_toffoli_gate(
        *[f'ctrl_{i}' for i in range(n_half_ctrls)],
        'targ_a')
    )

    dag.add_gate(multi_toffoli_gate(
        *[f'ctrl_{i}' for i in range(n_half_ctrls, n_ctrls)],
        'targ_b')
    )

    return compile_qcb(dag, height, width, *externs, multi_toffoli, **kwargs) 

In [5]:
initial_size = 9
mt_4 = multi_toffoli(4, initial_size, initial_size, ccz_factory_l1, init=True, toffoli=ccz_toffoli_gate,
                            allocator_kwargs={'over_allocate':True}
        )

NameError: name 'ccz_toffoli_gate' is not defined

In [95]:
initial_size = 9
mt_4 = multi_toffoli(4, initial_size, initial_size, ccz_factory_l1, init=True, toffoli=ccz_toffoli_gate,
                            allocator_kwargs={'over_allocate':True}
        )

i = 3
size = max(initial_size + ccz_factory_l1.width + 2, (1 << i) + 1)

mt_8 = nested_multi_toffoli(
                    i,
                    initial_size + 3 * i,
                    size,
                    ccz_factory_l1,
                    multi_toffoli=mt_4,
                    toffoli_gate = ccz_toffoli_gate
                )
prev_size = size

In [96]:
i = 4
size = max(prev_size + ccz_factory_l1.width + 2, (1 << i) + 1)

mt_16 = nested_multi_toffoli(
                    i,
                    initial_size + 3 * i,
                    size * 2,
                    ccz_factory_l1,
                    multi_toffoli=mt_8,
                    toffoli_gate = ccz_toffoli_gate
                )
prev_size = size

In [97]:
i = 5
size = max(prev_size + ccz_factory_l1.width + 2, (1 << i) + 1)

mt_32 = nested_multi_toffoli(
                    i,
                    initial_size + 3 * i,
                    size * 2,
                    ccz_factory_l1,
                    multi_toffoli=mt_16,
                    toffoli_gate = ccz_toffoli_gate
                )

In [91]:
size

27

In [93]:
prev_size

21

In [90]:
i = 4
size = max(prev_size + ccz_factory_l1.width + 2, (1 << i) + 1)

mt_64 = nested_multi_toffoli(
                    i,
                    initial_size + 3 * i,
                    size * 2,
                    ccz_factory_l1,
                    multi_toffoli=mt_32,
                    toffoli_gate = ccz_toffoli_gate
                )

AllocatorError: Could not allocate extern: EXTERN: <CCZ_Factory> : (<ctrl_1>, <ctrl_0>, <targ>), no valid placement

In [36]:
mt_4.dag.physical_externs, mt_4.n_cycles(), mt_4.space_time_volume()

([EXTERN: <CCZ_Factory> : (<ctrl_1>, <ctrl_0>, <targ>)], 791, 28434)

In [48]:
initial_size = 14
ccz_toffoli_gate = partial(cnot_network.toffoli_from_ccz, ccz_factory=ccz_factory_l3)

mt_4 = multi_toffoli(4, initial_size, initial_size, ccz_factory_l3, init=True, toffoli=ccz_toffoli_gate,
                            allocator_kwargs={'over_allocate':True}
        )

In [6]:
i = 3
initial_size = 14
size = max(initial_size + 4 * i, (1 << i) + 1)
mt_8 = nested_multi_toffoli(
                    i,
                    initial_size + 3,
                    size,
                    ccz_factory_l3,
                    multi_toffoli=mt_4,
                    toffoli_gate = ccz_toffoli_gate
                )

NameError: name 'mt_4' is not defined

In [50]:
mt_8.n_cycles()

46706

In [40]:
size

26

In [27]:
qcb = nested_multi_toffoli(
                n,
                initial_size + 4 * i,
                size + 4,
                ccz_factory,
                multi_toffoli=prev_mt,
                toffoli_gate = ccz_toffoli_gate
            )

In [31]:
ccz_factory_l3.width

11

In [29]:
mt[4].width

9

In [ ]:
initial_sizes = (9, 12, 14)
repeats = 5

import sys
sys.setrecursionlimit(5000)

results = {'distillation':[], 'n_ctrl': [], 'height':[], 'runtime':[], 'volume':[], 'delay':[]}

for distillation, ccz_factory, initial_size in zip((1, 2, 3), ccz_factories, initial_sizes):
    
    ccz_toffoli_gate = partial(cnot_network.toffoli_from_ccz, ccz_factory=ccz_factory)

    for rep in range(repeats):
    
        mt_4 = multi_toffoli(4, initial_size, initial_size, ccz_factory, init=True, toffoli=ccz_toffoli_gate)
        mt = mt_4
        prev_size = initial_size
        
        for i in range(3, 9):
            try:
                if distillation == 3 and i >= 8:
                    continue
                
                n = 1 << i
                print(f"\rCompiling {distillation}: Repeat {rep} : {n}" + " " * 10, end='', flush=True)
                prev = n // 2
    
                size = max(prev_size + ccz_factory.width + 2, (1 << i) + ccz_factory.width + 2)
    
                
                prev_size = size
                qcb = nested_multi_toffoli(
                    n,
                    initial_size + 4 * i,
                    size,
                    ccz_factory,
                    multi_toffoli=mt,
                    toffoli_gate = ccz_toffoli_gate
                )
                results['distillation'].append(distillation)
                results['n_ctrl'].append(2 ** i)
                results['runtime'].append(qcb.n_cycles())
                results['volume'].append(qcb.space_time_volume())
                mt = qcb
            except:
                pass

Compiling 3: Repeat 4 : 128          

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": "Helvetica",
})

#f, ax = plt.subplots(figsize=(7, 7))


sbs.set_theme()
sbs.set_style("dark")
sbs.set_style("whitegrid", {'grid.linestyle': '-.'})

#locmin = mticker.LogLocator(base=10, subs=np.arange(0.1,1,0.1),numticks=20)  
#ax.yaxis.set_minor_locator(locmin)
#ax.yaxis.set_minor_formatter(mticker.NullFormatter())
#plt.grid(True,which="both")  


marks = ['', '.', 'x', '*']
linestyles = ['-', '--', ':']

dependent = 'volume'
independent = 'n_ctrl'
#fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7, 7))

ax = [
    plt.subplot2grid((1, 2), (0, 0), colspan=1),
    plt.subplot2grid((1, 2), (0, 1), colspan=1)
]

subplot = ax[0]
kwargs = {
'errorbar':('ci', 100)
}

for idx, target in enumerate((1, 2, 3)):
    sbs.lineplot(
        ax=subplot,
        **plt_data(
            results['distillation'],
            results[independent],
            results[dependent],
            target=target
        ),
        label=f'DAG$_{target}$',
        linestyle=linestyles[idx],
        marker='^',
        markersize=9,
        color=sbs.color_palette()[0],
        **kwargs
    )


subplot.legend_.remove()
subplot.yaxis.set_label_position("left")
subplot.yaxis.tick_left()
subplot.set_yscale('log')
subplot.set_ylabel('Space-Time Volume')

dependent = 'runtime'

subplot = ax[1]
for idx, target in enumerate((1,2,3)):
    sbs.lineplot(
        ax=subplot,
        **plt_data(
            results['distillation'],
            results[independent],
            results[dependent],
            target=target
        ),
        label=f'$T^{{8}}_{{{target - 1}}} \\rightarrow CCZ^{{1}}_{{{target}}}$',
        linestyle=linestyles[idx],
        marker='^',
        markersize=9,
        color=sbs.color_palette()[0],
        **kwargs
    )

    
subplot.legend_.remove()
subplot.yaxis.set_label_position("right")
subplot.yaxis.tick_right()
subplot.set_yscale('log')
subplot.set_ylabel('Cycles')

fig = plt.gcf()
fig.set_size_inches(7, 7)

lines = linestyles * 3
labels = sum(([f'CCZ_{target}$', f'Extern$_{target}$', f'CCZ$_{target}$'] for target in range(3)), start=list())

handles, labels = subplot.get_legend_handles_labels()
plt.figlegend(handles, labels, loc='upper left', bbox_to_anchor=(0.43, 0.75))

fig.supxlabel('C$^{n}$X')

#fig.supxlabel('C$^{n}$X $=$ C$_a^{\\frac{n}{2}}$X C$_b^{\\frac{n}{2}}$X Toffoli$_{a, b} C$_b^{\\frac{n}{2}}$X C$_a^{\\frac{n}{2}}$X')


#sbs.lineplot(x=results_extern['n_ctrl'], y=results_extern['volume'], label="Toffoli Extern")
#sbs.lineplot(x=results_ccz['n_ctrl'], y=results_ccz['volume'], label='Toffoli Factory', **ccz_style, marker='^')


#plt.title('Toffoli Runtimes for Various QCB Sizes')
#plt.xlabel('C$^{n}$X')
#plt.ylabel('Space-Time Volume')

#plt.yscale('log')

#plt.legend()

#plt.savefig('nested_multi_toffolis.pdf')